# py4DSTEM - Tutorial on constructing probe templates
---

### Introduction

This is a tutorial notebook for constructing STEM probe template images. The goal is to measure or approximate the image of a STEM probe in vacuum (i.e. no scattering from a sample), and shift it to the origin. These "probe templates" can be used for:
1. Bragg disk detection using correlation.
2. Bragg disk detection using machine learning networks.
3. Calibration of the probe semiangle, ellipticity, or other probe geometric parameters.
4. Inital probe intensity distribution for ptychographic reconstructions.

There are 4 methods of constructing a probe template covered in this tutorial.  In order from best to worst, they are:
1. Immediately after completing on or more 4D-STEM experiments: Move the sample out of the way, and record of a series of images of the STEM probe under the exact same imaging conditions, with no scanning.
2. Average a subset of images from an experimental dataset, where the probe passes through vacuum.
3. Average a subset of images from an experimental dataset, where the probe passes though the thinnest posisble region of the sample.
4. Construct a synthetic probe image.

### Information

This tutorial was written by Colin Ophus (cophus@gmail.com).
The 4D-STEM datasets used here were collected by:
1. Clarissa Bhargava
2. Sean Mills
3. Karen Bustillo
4. Shiteng Zhao

### Version
Last updated on 2021-06-01 with py4DSTEM version 0.12.0.

# Download the datasets used in this tutorial

You can download the 4 tutorial datasets here: 
1. [part 1](https://drive.google.com/file/d/1QTcSKzZjHZd1fDimSI_q9_WsAU25NIXe/view?usp=sharing)
2. [part 2](https://drive.google.com/file/d/1iBSANFQT9eacpn7aAE6PmFdIRYQxenGg/view?usp=sharing)
3. [part 3](https://drive.google.com/file/d/1p9GaV0k628_afbdqFW62rHObcjQ09qyl/view?usp=sharing)
4. [part 4](https://drive.google.com/file/d/1sUrPEgM1wWyTh-LJ30lGUhcXklHj6ajC/view?usp=sharing). 

Place the file somewhere on your filesystem, then in the cell immediately after this one, update the variable `filepath_input` to reflect that path to the file.  Then update `filepath_output` to set wherever you want to save the outputs.

In [ ]:
# Set the file paths for the input datasets
file_path_example_01 = 'c:/Data/calibrationData_simulatedAuNanoplatelet_binned.h5'

In [ ]:
# Initialization - import the needed packages
import py4DSTEM